In [1]:
import numpy as np
import tensorflow as tf
from kfmodel2 import KFModel2
import os
from pykalman import KalmanFilter
from KF_params_MLE import tune_KF_params_MLE

In [2]:
with open('data.txt') as f:
    first = f.readline()
    first = first.strip('\n')
    temp = first.split(' ')
    T = int(temp[0])
    o_dim = int(temp[1])
    s_dim = int(temp[2])
    o_matrix = np.zeros((T, o_dim), np.float32)
    for i in range(T):
        temp = f.readline().strip('\n').split(' ')
        for j in range(s_dim):
            o_matrix[i,j] = float(temp[j])
    s_matrix = np.zeros((T, s_dim), np.float32)
    for i in range(T):
        temp = f.readline().strip('\n').split(' ')
        for j in range(o_dim):
            s_matrix[i,j] = float(temp[j])

In [3]:
T_train = 1000
T_test = 1000

In [23]:
A_2, b_2, Sig_2, A_3, b_3, Sig_3 = tune_KF_params_MLE(s_matrix[:T_train-1,:],\
                                                 s_matrix[1:T_train,:], o_matrix[1:T_train,:])
kf_orig = KalmanFilter(\
                       initial_state_mean =\
                           np.matmul(A_2, np.transpose(s_matrix[T_train-1,:])) + b_2.reshape(-1),\
                       initial_state_covariance = Sig_2,\
                       transition_matrices = A_2, \
                       transition_covariance = Sig_2, \
                       transition_offsets = b_2.reshape(-1),\
                       observation_matrices = A_3,\
                       observation_covariance = Sig_3,\
                       observation_offsets = b_3.reshape(-1)\
                      )
measurements = o_matrix[T_train:T_train+T_test,:]
(est_s, _) = kf_orig.filter(measurements)
print(np.mean(np.linalg.norm(est_s - s_matrix[T_train:T_train+T_test,:] , axis = 1)), end = '')
print(' / ', end = '')
print(np.mean(np.linalg.norm(s_matrix[T_train:T_train+T_test], axis = 1)))
print('mean consecutive diff: ', end='')
print(np.mean(np.linalg.norm(s_matrix[T_train+1:T_train+T_test] - \
                             s_matrix[T_train:T_train+T_test-1], axis = 1)))

est_o = np.matmul(np.matmul(est_s[:-1,:], A_2.T), A_3.T)
print(np.mean(np.linalg.norm(est_o - o_matrix[T_train+1:T_train+T_test,:] , axis = 1)), end = '')

2.2566873418 / 2.23837
mean consecutive diff: 3.11463
2.2074319032

In [5]:
class KFModelDataset(object):
    def __init__(self, train_S_t_minus_1, train_S_t, train_O_t):
        self.train_S_t_minus_1 = train_S_t_minus_1
        self.train_S_t = train_S_t
        self.train_O_t = train_O_t
    def random_batch(self, batch_size):
        index = np.random.choice(np.arange(len(self.train_S_t)),batch_size, False)
        return self.train_S_t_minus_1[index,:], self.train_S_t[index,:], self.train_O_t[index,:]
kfmodeldataset = KFModelDataset(s_matrix[:T_train-1,:], s_matrix[1:T_train,:], o_matrix[1:T_train,:])

In [6]:
log_dir = './log/'
os.popen('rm '+log_dir+'*')
h_dim = 100
minibatch_size = 128
kfmodel2 = KFModel2(s_dim, o_dim, h_dim, minibatch_size, 1e-3, log_dir)
iteration = 300
for epoch in range(1000):
    S_recons_loss_train, S_pred_loss_train, O_recons_loss_train,\
    O_pred_loss_train, D1_cross_ent_train, D2_cross_ent_train = 0.,0.,0.,0.,0.,0.
    global_step = tf.contrib.framework.get_or_create_global_step()
    # in each epoch 500 iterations
    for i in range(iteration):
        train_S_t_minus_1, train_S_t, train_O_t = kfmodeldataset.random_batch(minibatch_size)
        S_recons_loss_value, S_pred_loss_value, O_recons_loss_value,\
        O_pred_loss_value, D1_cross_ent_value, D2_cross_ent_value,\
        summary = kfmodel2.update_params(train_S_t_minus_1, train_S_t, train_O_t)
            
            
        S_recons_loss_train += S_recons_loss_value
        S_pred_loss_train += S_pred_loss_value
        O_recons_loss_train += O_recons_loss_value
        O_pred_loss_train  += O_pred_loss_value
        D1_cross_ent_train += D1_cross_ent_value
        D2_cross_ent_train += D2_cross_ent_value   
        kfmodel2.train_writer.add_summary(summary, global_step.eval(kfmodel2.sess))
    
    S_recons_loss_train /= iteration
    S_pred_loss_train /= iteration
    O_recons_loss_train /= iteration
    O_pred_loss_train  /= iteration
    D1_cross_ent_train /= iteration
    D2_cross_ent_train /= iteration
    
    print("step: {},\n \
    S_recons_loss: {:.2f},\tS_pred_loss: {:.2f},\n \
    O_recons_loss: {:.2f},\tO_pred_loss: {:.2f},\n \
    D1_cross_ent: {:.2f},\tD2_cross_ent: {:.2f}\n \
    ------------------------------------------------------------------"\
          .format(global_step.eval(kfmodel2.sess),S_recons_loss_train, S_pred_loss_train,\
                  O_recons_loss_train, O_pred_loss_train, D1_cross_ent_train, D2_cross_ent_train))


step: 1200,
     S_recons_loss: 0.77,	S_pred_loss: 2.19,
     O_recons_loss: 0.61,	O_pred_loss: 2.16,
     D1_cross_ent: 0.00,	D2_cross_ent: 0.00
     ------------------------------------------------------------------
step: 2400,
     S_recons_loss: 0.49,	S_pred_loss: 1.88,
     O_recons_loss: 0.28,	O_pred_loss: 1.79,
     D1_cross_ent: 0.00,	D2_cross_ent: 0.00
     ------------------------------------------------------------------
step: 3600,
     S_recons_loss: 0.43,	S_pred_loss: 1.56,
     O_recons_loss: 0.22,	O_pred_loss: 1.44,
     D1_cross_ent: 0.00,	D2_cross_ent: 0.00
     ------------------------------------------------------------------
step: 4800,
     S_recons_loss: 0.38,	S_pred_loss: 1.37,
     O_recons_loss: 0.20,	O_pred_loss: 1.24,
     D1_cross_ent: 0.00,	D2_cross_ent: 0.00
     ------------------------------------------------------------------
step: 6000,
     S_recons_loss: 0.33,	S_pred_loss: 1.22,
     O_recons_loss: 0.18,	O_pred_loss: 1.10,
     D1_cross_ent: 0.00,	D

KeyboardInterrupt: 

In [24]:
last_s_p = np.asarray(kfmodel2.sess.run([kfmodel2.S_p_t],\
                                        {kfmodel2.S_t_minus_1: s_matrix[T_train-3:T_train-1, :],\
                                         kfmodel2.S_t: s_matrix[T_train-2:T_train, :],\
                                         kfmodel2.O_t: o_matrix[T_train-2:T_train , :]
                                        }))[0][1,:]

A_1, b_1, R_1, A_2, b_2, R_2 = kfmodel2.sess.run([kfmodel2.A_1, kfmodel2.b_1, kfmodel2.R_1,\
                                                  kfmodel2.A_2, kfmodel2.b_2, kfmodel2.R_2])
Sig_1 = np.matmul(R_1, R_1.T)
Sig_2 = np.matmul(R_2, R_2.T)
b_1 = b_1.reshape([-1])
b_2 = b_2.reshape([-1])
kf = KalmanFilter(initial_state_mean = np.matmul(A_1,last_s_p.T) + b_1, \
                  initial_state_covariance = Sig_1, \
                  transition_matrices = A_1, \
                  transition_offsets = b_1,
                  transition_covariance = Sig_1, \
                  observation_matrices = A_2,\
                  observation_offsets = b_2,
                  observation_covariance = Sig_2)


In [26]:
est_O_p_t = kfmodel2.sess.run(kfmodel2.O_p_t, {kfmodel2.O_t: o_matrix[T_train:T_train+T_test, :]})
measurements = np.asarray(est_O_p_t)
(est_S_p_t, est_S_p_t_covariances) = kf.filter(measurements)
est_S_t = kfmodel2.decode_S_p_t(est_S_p_t)
print(np.mean(np.linalg.norm(est_S_t - s_matrix[T_train:T_train+T_test,:] , axis = 1)))

est_O_p_t_plus_1 = np.matmul(np.matmul(est_S_p_t[:-1,:], A_1.T), A_2.T)
est_O_t_plus_1 = kfmodel2.decode_O_p_t(est_O_p_t_plus_1)
print(np.mean(np.linalg.norm(est_O_t_plus_1 - o_matrix[T_train+1:T_train+T_test,:] , axis = 1)))

est_O_p_t = np.matmul(est_S_p_t, A_2.T)
est_O_t = kfmodel2.decode_O_p_t(est_O_p_t)
print(np.mean(np.linalg.norm(est_O_t - o_matrix[T_train:T_train+T_test,:] , axis = 1)))

2.28364
3.58192
1.15767


In [28]:
R_2

array([[ 0.00985505,  0.00540362,  0.00216095, ..., -0.00723095,
        -0.01933458, -0.00401116],
       [ 0.00434473,  0.01017877, -0.00039254, ...,  0.00825434,
         0.00963332, -0.00586   ],
       [ 0.0030291 , -0.00823032,  0.01181617, ..., -0.00134031,
        -0.00426158, -0.00446487],
       ..., 
       [ 0.00173561, -0.00316625,  0.00744166, ..., -0.00143443,
        -0.017617  ,  0.0032794 ],
       [-0.0090737 , -0.00212268,  0.00265729, ..., -0.01303044,
         0.01403006,  0.00953421],
       [ 0.00532057, -0.00948359,  0.00065001, ..., -0.00056666,
         0.01776673, -0.00010234]], dtype=float32)